# NFL Score prediction based on Home Vs Away Teams

In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import re

from scipy import stats
from pandas.plotting import register_matplotlib_converters

from statsmodels.nonparametric.smoothers_lowess import lowess
from pykalman import KalmanFilter

from sklearn.linear_model import LinearRegression

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.color import lab2rgb
import sys

from skimage import io, color
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier


In [2]:
filename = 'games_2017.csv'
df = pd.read_csv(filename)
df['home_win'] = df['homescore']>df['awayscore']

In [3]:
from collections import defaultdict
won_last = defaultdict(int)
df2 = df
df2['HomeLastWin'] = 0
df2['VisitorLastWin'] = 0
for index, row in df.iterrows():
    home_team = row["home"]
    visitor_team = row["away"]
    row["HomeLastWin"] = won_last[home_team]
    row["VisitorLastWin"] = won_last[visitor_team]
    df2.iloc[index] = row
    won_last[home_team] = row["home_win"]
    won_last[visitor_team] = not row["home_win"]

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
clf = DecisionTreeClassifier(random_state=1)
y_true = df2['home_win']
X_previouswins = df2[["HomeLastWin", "VisitorLastWin"]].values
scores = cross_val_score(clf, X_previouswins, y_true, scoring='accuracy', cv=100)
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Accuracy: 58.4%


In [5]:
X = df[['home','away','homescore','awayscore']]
y = df['home_win']

In [6]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y)
X_valid, y_valid

(    home away  homescore  awayscore
 195   TB  DET         21         24
 21   JAX  TEN         16         37
 246  PHI  DAL          0          6
 178  TEN  HOU         24         13
 179  NYJ   KC         38         31
 ..   ...  ...        ...        ...
 5    DET  ARI         35         23
 247   LA   SF         13         34
 205  JAX  SEA         30         24
 116  WAS  DAL         19         33
 133  CHI   GB         16         23
 
 [64 rows x 4 columns], 195    False
 21     False
 246    False
 178     True
 179     True
        ...  
 5       True
 247    False
 205     True
 116    False
 133    False
 Name: home_win, Length: 64, dtype: bool)

In [7]:
import seaborn
from scipy.stats import poisson,skellam
import statsmodels.api as sm
import statsmodels.formula.api as smf

#### Predicting Football Scores based on home and away te https://dashee87.github.io/football/python/predicting-football-results-with-statistical-modelling/


In [8]:
df_train = pd.concat([X_train,y_train],axis=1,sort=False)
df_valid = pd.concat([X_valid,y_valid],axis=1,sort=False)

In [10]:
poisson_model = smf.glm(formula="homescore ~ home + away + awayscore", data=df_train, 
                        family=sm.families.Poisson()).fit()

In [11]:
poisson_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:              homescore   No. Observations:                  192
Model:                            GLM   Df Residuals:                      128
Model Family:                 Poisson   Df Model:                           63
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -788.51
Date:                Sat, 30 Nov 2019   Deviance:                       657.53
Time:                        16:40:01   Pearson chi2:                     597.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       2.9929      0.135     22.217      0.000       2.729       3.257
home[T.ATL]    -0.0218      0.123     -0.177      0.859      -0.263       0.219
home[T.BAL]     0.0733      0.127      0.575      0.565      -0.176       0.323
home[T.BUF]    -0.1779      0.131     -1.353      0.176      -0.436       0.080
home[T.CAR]     0.0792      0.125      0.633      0.527      -0.166       0.324
home[T.CHI]    -0.1301      0.141     -0.922      0.356      -0.407       0.146
home[T.CIN]    -0.4210      0.156     -2.703      0.007      -0.726      -0.116
home[T.CLE]    -0.6307      0.168     -3.756      0.000      -0.960      -0.302
home[T.DAL]    -0.0938      0.131     -0.716      0.474      -0.350       0.163
home[T.DEN]     0.2499      0.132      1.900      0.057      -0.008       0.508
home[T.DET]     0.1404      0.130      1.081      0.280      -0.114       0.395
home[T.GB]     -0.0587      0.157     -0.374      0.708      -0.366       0.249
home[T.HOU]     0.2077      0.127      1.633      0.102      -0.042       0.457
home[T.IND]    -0.3299      0.129     -2.552      0.011      -0.583      -0.077
home[T.JAX]     0.2613      0.138      1.896      0.058      -0.009       0.531
home[T.KC]     -0.0908      0.133     -0.681      0.496      -0.352       0.171
home[T.LA]      0.2632      0.152      1.737      0.082      -0.034       0.560
home[T.LAC]    -0.1138      0.136     -0.840      0.401      -0.379       0.152
home[T.MIA]    -0.2118      0.134     -1.579      0.114      -0.475       0.051
home[T.MIN]    -0.0071      0.135     -0.053      0.958      -0.272       0.257
home[T.NE]      0.3127      0.125      2.492      0.013       0.067       0.559
home[T.NO]      0.2617      0.123      2.120      0.034       0.020       0.504
home[T.NYG]    -0.4335      0.146     -2.961      0.003      -0.721      -0.147
home[T.NYJ]     0.0392      0.138      0.285      0.776      -0.231       0.309
home[T.OAK]    -0.1563      0.160     -0.980      0.327      -0.469       0.156
home[T.PHI]     0.2916      0.118      2.473      0.013       0.060       0.523
home[T.PIT]     0.1899      0.130      1.466      0.143      -0.064       0.444
home[T.SEA]    -0.0782      0.133     -0.588      0.556      -0.339       0.182
home[T.SF]     -0.2001      0.132     -1.515      0.130      -0.459       0.059
home[T.TB]      0.1234      0.133      0.931      0.352      -0.136       0.383
home[T.TEN]     0.1765      0.140      1.260      0.208      -0.098       0.451
home[T.WAS]     0.0476      0.125      0.382      0.703      -0.197       0.292
away[T.ATL]    -0.0660      0.132     -0.500      0.617      -0.325       0.193
away[T.BAL]     0.1525      0.150      1.014      0.311      -0.142       0.447
away[T.BUF]    -0.0361      0.135     -0.268      0.789      -0.300       0.228
away[T.CAR]    -0.0896      0.133     -0.67

In [12]:
df_test = pd.DataFrame(data={'home': 'NE','away':'KC','awayscore':1},index=[1])
df_test
poisson_model.predict(df_test)
# poisson_model.predict(df_valid)

1    23.132014
dtype: float64

In [13]:
df_test = pd.DataFrame(data={'home': 'KC','away':'NE','awayscore':1},index=[1])
df_test
poisson_model.predict(df_test)

1    11.41855
dtype: float64